In [1]:
# 

import numpy as np
import matplotlib.pyplot as plt4
%matplotlib notebook 



class LinearRegressionModule():
    def __init__(self, iteration_count, learning_alpha):
        self.num_iterations=iteration_count
        self.learning_rate=learning_alpha
    
############################# Core Functionalities   ###############################

################################## Direct Methods ###################################
        
    def direct_method_overdetermined(self,X,Y):
        W=(np.linalg.inv((X.T).dot(X)).dot(X.T)).dot(Y)
        return W
    def direct_method_underdetermined(self,X,Y):
        W=((X.T).dot(np.linalg.inv((X).dot(X.T)))).dot(Y)
        return W
    
################################## Iterative Methods functionalites ###################################   
    
    def grad_descent(self,X,Y,W):
        N=X.shape[0]
        n=X.shape[1]           
        W=W-((X.T).dot(X.dot(W)-Y))*(self.learning_rate/N)            ## Batch gradient Descent      
        return W  
    
    
    def predict_Y(self,X,W):
        return X.dot(W)
    
   
    def stochastic_descent(self,X,Y,W,alpha):
        N=X.shape[0]
        #for j in range (self.num_iterations):
        for i in range(N):
            W=W-(alpha/N)*((X[[i],:]).T).dot(((X[[i],:]).dot(W)-Y[i]))                
        return W
    
    def cost_function(self,X,Y,W):
        N=X.shape[0]
        n=X.shape[1]
        j=0
        j=(((X.dot(W)-Y).T).dot((X.dot(W)-Y)))*(1/(2*N))        
        return j
    
   ################################## UtilitY functionalites ###################################  
    
    def write_weights_to_file(self,file_name, W):
        param_file=open(file_name,"w")
        for i in range(0,W.shape[0]):
            param_file.write(str(W[i]))
            param_file.write("\n")
        param_file.close()
    def zscore_normalization(self,X):
        N=X.shape[0]
        n=X.shape[1]        
        print("N,n=",N,n)        
        data_norm=X        
        mean=[]
        sigma=[]        
        mean=np.mean(data_norm,axis=0)
        sigma=np.std(data_norm,axis=0)        
        for j in range(data_norm.shape[1]-1):
            for i in range(data_norm.shape[0]):
                data_norm[i][j] = (data_norm[i][j]-mean[j])/(sigma[j])
        return data_norm
    
    def minmax_normalization(self,X):
        N=X.shape[0]
        n=X.shape[1]        
        print("N,n=",N,n)        
        data_norm=X
        for j in range(data_norm.shape[1]):
            min = np.min(data_norm[:,j])
            max = np.max(data_norm[:,j])
            data_norm[:,j] = (data_norm[:,j] - np.min(data_norm[:,j])) / (np.max(data_norm[:,j]) - np.min(data_norm[:,j]))
            
        return data_norm
                
    def hold_out_validation(self,X,Y,alpha_min,alpha_max,alpha_size):
        N=X.shape[0]
        n=X.shape[1]
        mean_square_error = []
        std_deviation     = []
        alpha_arrayay       = []
        
 
        for alpha in np.linspace(alpha_min,alpha_max,alpha_size):            
            error_sum=0.0
            for i in range(50):
                #error_sum=0.0
                data= np.random.permutation(X)
                zero_column_train=tuple(np.ones(((int)(0.7*N),1)))
                zero_column_test=tuple(np.ones(((int)(0.3*N)+1,1)))
                
                X_train=data[0:int(0.7*N),0:n-1]
                Y_train=data[0:int(0.7*N),n-1:n]
                X_test=data[int(0.7*N):N,0:n-1]                
                Y_test=data[int(0.7*N):N,n-1:n]               
                X_train=np.hstack((zero_column_train,X_train))
                X_test=np.hstack((zero_column_test,X_test))               
                norm=1000000*1000000 # a very high number
                W_old=np.zeros((n,1))
                W_new=W_old
                self.learning_rate=alpha
                no_of_iterations=0
                while(norm > 0.001 and no_of_iterations < 20000):
                    W_new=self.grad_descent(X_train,Y_train,W_old)
                    norm=np.linalg.norm(W_new-W_old)
                    W_old=W_new
                    no_of_iterations+=1
                cost         = self.cost_function(X_test,Y_test,W_new)
                error_sum    = error_sum + cost
                Y_predict    = self.predict_Y(X_test,W_new)
                diff_vec     = Y_test - Y_predict
                std_dev      = np.std(diff_vec)
                
            std_deviation.append(std_dev/50)    
            mean_square_error.append(error_sum/50)
            alpha_arrayay.append(alpha)        
        min_index     = mean_square_error.index(min(mean_square_error))
        min_std_min   = min(std_deviation) 
        return min(mean_square_error),alpha_arrayay[min_index],min_std_min
            
       
    def k_fold_cross_validation(self,dataset,alpha_min,alpha_max,step_size):
        
        N=dataset.shape[0]
        n=dataset.shape[1]
        print(N,n)
        erroe_sum=0.0
        for i in range(0,30):       
            data= np.random.permutation(dataset)
            X_train=data[0:int(0.7*N),0:n-1]
            Y_train=data[0:int(0.7*N),n-1:n]
            X_test=data[int(0.7*N):N,0:n-1]
            Y_test=data[int(0.7*N):N,n-1:n]

            alpha_kfold=self.k_fold_validation(X_train,Y_train,alpha_min,alpha_max,step_size)
            print(" Alpha minimum for K-Fold for iteration no",i+1," is ",alpha_kfold)
            W_old=np.zeros((n-1,1))
            W_new=W_old
            
            norm=100000*100000
            self.learning_rate=alpha_kfold
            no_of_iterations=0
            while(norm > 0.001 and no_of_iterations < 1000):
                W_new=self.grad_descent(X_train,Y_train,W_old)
                norm=np.linalg.norm(W_new-W_old)
                W_old=W_new
                no_of_iterations+=1
            pred_err=self.cost_function(X_test,Y_test,W_new)
            erroe_sum=erroe_sum + pred_err
            print(erroe_sum) 
                
        return erroe_sum/30
        
    
    def k_fold_validation(self,X,Y,alpha_min,alpha_max,steps):
        dataset=np.hstack((X,Y))
        N=dataset.shape[0]        
        fold=N//5    ## K is taken as 5
        n=dataset.shape[1]        
        k=5
        k1=dataset[0:fold,:]
        k2=dataset[fold:2*fold,:]
        k3=dataset[2*fold:3*fold,:]
        k4=dataset[3*fold:4*fold,:]
        k5=dataset[4*fold:5*fold,:]
        fold_list=[k1,k2,k3,k4,k5]
        
        
        mean_square_error=[]
        alpha_array=[]
        
        for alpha in np.linspace(alpha_min,alpha_max,steps):
            erroe_sum=0.0
            self.learning_rate=alpha
            for i in range(k):
                tmp=np.zeros((1,n))
                data_test=fold_list[i]
                X_test=data_test[:,0:n-1]
                Y_test=data_test[:,n-1:n]
                for j in range(k):
                    if(j!=i):
                        tmp=np.vstack((tmp,fold_list[j]))
                data_train=tmp[1:,:]
                X_train=data_train[:,0:n-1]
                Y_train=data_train[:,n-1:n]
                W_old=np.ones((n-1,1),dtype=float)
                W_new=W_old
                norm=100000*100000
                no_of_iterations=0
                while(norm > 0.001 and no_of_iterations < 1000):
                    W_new=self.grad_descent(X_train,Y_train,W_old)
                    norm=np.linalg.norm(W_new-W_old)
                    W_old=W_new
                    no_of_iterations+=1
                
                cost=self.cost_function(X_test,Y_test,W_new)               
                erroe_sum=erroe_sum + cost

            mean_square_error.append(erroe_sum/k)            
            alpha_array.append(alpha)        
        index=mean_square_error.index(min(mean_square_error))
        return alpha_array[index]
    
    

In [2]:
#Analyze Wine quality data set using linear regression (download data from UCI web
#repository)
#(a) Analyze the data with normalization and without normalization.
#(b) Describe how you applied normalization techniques on training and testing data.
#(c) Apply k fold cross validation and hold out method.
#(d) Assess the performance of the model.
#(e) Report search space & the values of the hyperparameters and the parameters of
#the model.
#(f) Apply batch as well as online optimization algorithms and compare their perfor-
#mance using statistical measures. Compare the time taken by the two algorithms


In [3]:
# Steps followed
   # 1. Normalize the data using min max normalizer
   # 1. Find the weights and  cost function using direct method.
   # 2. Find out the alpha value from hold out method and K fold iteration method
   # 3. Use this alpha for gradient descent and find out whether the cost function coverges
# Search space for alpha parameter is selected on trail and error basis as 0.01 to0.040
# The best alpha was found out from hold out iteration as 0.035 and was selected as the hyper 
# parameter for gradient descent method
# Weights obtained from both direct and gradient descent methods are saved in output folder
# as .csv files
# The final cost function and time taken for each methods are outputted from the cell


In [4]:
from timeit import default_timer as timer

input_data5 = np.genfromtxt("data/winequality-red.csv",delimiter=';',dtype=float)
lrm = LinearRegressionModule(50,0.035)
#print(input_data5)

input_data5_processed = np.delete(input_data5, 0, axis=0)
N=input_data5_processed.shape[0]

data_normalized=lrm.minmax_normalization(input_data5_processed)
#print(data_normalized)
n=input_data5_processed.shape[1]
ones_row=tuple(np.ones((N,1)))
X=data_normalized[:, 0:n-1]  
X=np.hstack((ones_row,X))
Y=data_normalized[:,n-1:n]
N=X.shape[0]
n=X.shape[1]
W_current=np.ones((n,1))
print("Shape of normalised:",N,n)
W_new       =W_current
start = timer()
W_new       =lrm.direct_method_overdetermined(X,Y)
stop = timer()
total_time = stop-start
print("time taken for direct method = ", total_time)
print("Parameters after direct method")
cost = lrm.cost_function(X,Y,W_new)
print("Cost for direct method = " ,cost)
lrm.write_weights_to_file("output/winequality-red-direct.csv", W_new)

W_current=np.ones((n,1))
iteration_no=0
norm=1000*10000
W_new=W_current
start = timer()
while(norm > 0.00001 and iteration_no < 100000):
    W_new=lrm.grad_descent(X,Y,W_current)    
    norm=np.linalg.norm(W_new-W_current)   
    W_current=W_new
    cost=lrm.cost_function(X,Y,W_new)   
    iteration_no+=1
stop = timer()
total_time = stop - start
print ("Total time taken for iterative method_batch_gradient (secs) = " ,total_time)
print("Parameters after iterative batch gradient method")
cost = lrm.cost_function(X,Y,W_new)
print("Cost for iterative batch method  = " ,cost)
lrm.write_weights_to_file("output/winequality-red-batch-iteration.csv",W_new)


W_current=np.zeros((n,1))
W_new=W_current
norm=100000*100000
start = timer()
while(norm > 0.0001):
    W_new=lrm.stochastic_descent(X,Y,W_current,0.10)
    norm=np.linalg.norm(W_new-W_current)
    W_current=W_new        
    end = timer()    
print("Total time taken for iterative method_stochastic_gradient (secs):",end-start)
print("Parameters after iterative stochastic gradient method")


cost = lrm.cost_function(X,Y,W_new)
print("Cost for iterative online method  = " ,cost)
lrm.write_weights_to_file("output/winequality-red-online-iteration.csv",W_new)






#print(input_data4)



N,n= 1599 12
Shape of normalised: 1599 12
time taken for direct method =  0.021725199999991673
Parameters after direct method
Cost for direct method =  [[0.00833534]]
Total time taken for iterative method_batch_gradient (secs) =  4.0573179999999525
Parameters after iterative batch gradient method
Cost for iterative batch method  =  [[0.0083666]]
Total time taken for iterative method_stochastic_gradient (secs): 97.46597409999998
Parameters after iterative stochastic gradient method
Cost for iterative online method  =  [[0.00843748]]


In [5]:
input_data5 = np.genfromtxt("data/winequality-red.csv",delimiter=';',dtype=float)
lrm = LinearRegressionModule(50,0.035)
#print(input_data5)

input_data5_processed = np.delete(input_data5, 0, axis=0)
N=input_data5_processed.shape[0]

data_normalized=lrm.minmax_normalization(input_data5_processed)
#print(data_normalized)
n=input_data5_processed.shape[1]
ones_row=tuple(np.ones((N,1)))
X=data_normalized[:, 0:n-1]  
X=np.hstack((ones_row,X))
Y=data_normalized[:,n-1:n]
N=X.shape[0]
n=X.shape[1]
W_current=np.ones((n,1))
print("Shape of normalised:",N,n)

a,b,c =lrm.hold_out_validation(X,Y,0.01,0.035,10)
print("Mean_square_Error_Minimum = ",a)
print("alpha_minimum = ",b)
print("Standard_Deviation_Minimum = ",c)

N,n= 1599 12
Shape of normalised: 1599 12
Mean_square_Error_Minimum =  [[0.00579887]]
alpha_minimum =  0.035
Standard_Deviation_Minimum =  0.0020456459401918853


In [19]:
input_data5 = np.genfromtxt("data/winequality-red.csv",delimiter=';',dtype=float)
lrm = LinearRegressionModule(50,0.035)
#print(input_data5)

input_data5_processed = np.delete(input_data5, 0, axis=0)
N=input_data5_processed.shape[0]

data_normalized=lrm.minmax_normalization(input_data5_processed)
#print(data_normalized)
n=input_data5_processed.shape[1]
ones_row=tuple(np.ones((N,1)))
X=data_normalized[:, 0:n-1]  
X=np.hstack((ones_row,X))
Y=data_normalized[:,n-1:n]
N=X.shape[0]
n=X.shape[1]
W_current=np.ones((n,1))
print("Shape of normalised:",N,n)
W_new       =W_current
validation_data=np.hstack((X,Y))
mse=lrm.k_fold_cross_validation(validation_data,0.001,0.035,20)
print("MSE:",mse)




N,n= 1599 12
Shape of normalised: 1599 12
1599 13
 Alpha minimum for K-Fold for iteration no 1  is  0.035
[[0.01249442]]
 Alpha minimum for K-Fold for iteration no 2  is  0.035
[[0.02524502]]
 Alpha minimum for K-Fold for iteration no 3  is  0.035
[[0.03892247]]
 Alpha minimum for K-Fold for iteration no 4  is  0.035
[[0.05118431]]
 Alpha minimum for K-Fold for iteration no 5  is  0.035
[[0.06495704]]
 Alpha minimum for K-Fold for iteration no 6  is  0.035
[[0.07862224]]
 Alpha minimum for K-Fold for iteration no 7  is  0.035
[[0.09110814]]
 Alpha minimum for K-Fold for iteration no 8  is  0.035
[[0.10417246]]
 Alpha minimum for K-Fold for iteration no 9  is  0.035
[[0.11685548]]
 Alpha minimum for K-Fold for iteration no 10  is  0.035
[[0.12950395]]
 Alpha minimum for K-Fold for iteration no 11  is  0.035
[[0.14325926]]
 Alpha minimum for K-Fold for iteration no 12  is  0.035
[[0.15559621]]
 Alpha minimum for K-Fold for iteration no 13  is  0.035
[[0.16896572]]
 Alpha minimum for K-Fo